<a href="https://colab.research.google.com/github/u5638928/u5638928-DataScience-GenAI-Submissions/blob/main/6_02_DNN_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://drive.google.com/uc?export=view&id=1xqQczl0FG-qtNA2_WQYuWePW9oU8irqJ)

# 6.02 Dense Neural Network (with PyTorch)
This will expand on our logistic regression example and take us through building our first neural network. If you haven't already, be sure to check (and if neccessary) switch to GPU processing by clicking Runtime > Change runtime type and selecting GPU. We can test this has worked with the following code:

In [ ]:
import torch

# Check for GPU availability
print("Num GPUs Available: ", torch.cuda.device_count())

Num GPUs Available:  1


Hopefully your code shows you have 1 GPU available! Next let's get some data. We'll start with another in-built dataset:

In [ ]:
# upload an in-built Python (OK semi-in-built) dataset
from sklearn.datasets import load_diabetes

import pandas as pd
import numpy as np

# import the data
data = load_diabetes()
data

{'data': array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
          0.01990749, -0.01764613],
        [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
         -0.06833155, -0.09220405],
        [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
          0.00286131, -0.02593034],
        ...,
        [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
         -0.04688253,  0.01549073],
        [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
          0.04452873, -0.02593034],
        [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
         -0.00422151,  0.00306441]]),
 'target': array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75., 142., 155., 225.,  59

We are working on a regression problem, with "structured" data which has already been cleaned and normalised. We can skip the usual cleaning/engineering steps. However, we do need to get the data into PyTorch:

In [ ]:
# Convert data to PyTorch tensors
X = torch.tensor(data.data, dtype=torch.float32)
y = torch.tensor(data.target, dtype=torch.float32).reshape(-1, 1) # Reshape y to be a column vector

Now our data is stored in tensors we can do train/test splitting as before (in fact we can use sklearn as before):

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch.Size([353, 10]) torch.Size([353, 1])
torch.Size([89, 10]) torch.Size([89, 1])


Now we can set up our batches for training. As we have a nice round 400 let's go with batches of 50 (8 batches in total). We'll also seperate the features and labels:

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)

Now its time to build our model. We'll keep it simple ... a model with an input layer of 10 features and then 2x _Dense_ (fully connected) layers each with 5 neurons and ReLU activation. Our output layer will be size=1 given this is a regression problem and we want a single value output per prediction.

This will be easier to understand if you have read through the logistic regression tutorial.

In [ ]:
import torch
import torch.nn as nn

# Define the model
class DiabetesModel(nn.Module):
    def __init__(self):
        super(DiabetesModel, self).__init__()
        # we'll set up the layers as a sequence using nn.Sequential
        self.layers = nn.Sequential(

            # first layer will be a linear layer that has 5x neurons
            # (5x sets of linear regression)
            # the layer takes the 10 features as input (i.e. 10, 5)
            nn.Linear(10, 5),

            nn.ReLU(), # ReLU activation

            # second linear layer again has 5 neurons
            # this time taking the input as the output of the last layer
            # (which had 5x neurons)
            nn.Linear(5, 5),

            nn.ReLU(), # ReLU again

            # last linear layer takes the output from the previous 5 neurons
            # this time its a single output with no activation
            # i.e. this is the predicitons (regression)
            nn.Linear(5, 1)
        )

    def forward(self, x):
        return self.layers(x) # pass the data through the layers

As before we need to create a model object, specify the loss (criterion) and an optimiser (which we cover next week):

In [ ]:
import torch.optim as optim

# Initialize the model, loss function, and optimizer
model = DiabetesModel()
criterion = nn.MSELoss() # MSE loss function
optimiser = optim.Adam(model.parameters(), lr=0.001)

Now we can train the model. Again, the logistic regression tutorial (6.01) may help you undertstand this:

In [ ]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training loop (example - you'll likely want to add more epochs)
epochs = 100 # 100 epochs

for epoch in range(epochs):
  # use the train_loader to pass the inputs (x) and targets (y)
  for inputs, targets in train_loader:
    # pass to the GPU (hopefully)
    inputs, targets = inputs.to(device), targets.to(device)

    # pass model to GPU as well
    model.to(device)

    model.train() # put the model object in train mode
    optimiser.zero_grad() # reset the gradiants
    outputs = model(inputs) # create outputs
    loss = criterion(outputs, targets) # compare with Y to get loss
    loss.backward() # backpropogate the loss (next week)
    optimiser.step() # # update the parameters based on this round of training

  # every 10 steps we will print out the current loss
    if (epoch+1) % 10 == 0: # modular arithmetic
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {round(loss.item(), 4)}')

Epoch [10/100], Loss: 23710.9648
Epoch [10/100], Loss: 22140.1797
Epoch [10/100], Loss: 23457.2227
Epoch [10/100], Loss: 25179.0547
Epoch [10/100], Loss: 28234.0449
Epoch [10/100], Loss: 26300.7109
Epoch [10/100], Loss: 24978.0664
Epoch [10/100], Loss: 25416.5449
Epoch [20/100], Loss: 22775.291
Epoch [20/100], Loss: 23511.6094
Epoch [20/100], Loss: 29007.5488
Epoch [20/100], Loss: 26157.4043
Epoch [20/100], Loss: 23944.4395
Epoch [20/100], Loss: 26790.7344
Epoch [20/100], Loss: 20413.9746
Epoch [20/100], Loss: 46202.3672
Epoch [30/100], Loss: 23736.9863
Epoch [30/100], Loss: 26637.0293
Epoch [30/100], Loss: 28107.375
Epoch [30/100], Loss: 22896.1562
Epoch [30/100], Loss: 22645.6621
Epoch [30/100], Loss: 25095.7285
Epoch [30/100], Loss: 25028.666
Epoch [30/100], Loss: 17878.6934
Epoch [40/100], Loss: 22190.7461
Epoch [40/100], Loss: 24890.8242
Epoch [40/100], Loss: 28786.5
Epoch [40/100], Loss: 24750.8691
Epoch [40/100], Loss: 26048.5645
Epoch [40/100], Loss: 23820.4688
Epoch [40/100], 

We can see loss is significantly lower at the end than it was at the start. However, it is also bouncing around a little still which suggests the model needs more training (100 epochs is not a lot in deep learning terms). However, let's evaluate as before:

In [ ]:
# Evaluation (example)
model.eval() # testing mode
mse_values = [] # collect the MSE scores

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs) # predict the test data

        # Calculate Mean Squared Error
        mse = criterion(outputs, targets) # calcualte mse for the batch
        mse_values.append(mse.item()) # add to the list of MSE values

# Calculate and print the average MSE
avg_mse = np.mean(mse_values)
print(f"Average MSE on test set: {avg_mse}")

Average MSE on test set: 25943.71484375


MSE looks expected given training (no obvious sign of overfitting). However, we probably can get better results with tuning and more epochs.

Let's run the loop again a little differently to collect the predicted values (y_hat) and actuals (y) and add them to a dataset for comparions:

In [ ]:
# Evaluation
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(targets.cpu().numpy())

# Create DataFrame
results_df = pd.DataFrame({'Predicted': np.array(predictions).flatten(), 'Actual': np.array(actuals).flatten()})
results_df

,Predicted,Actual
0,0.831514,219.0
1,0.831514,70.0
2,0.831514,202.0
3,0.831514,230.0
4,0.831514,111.0
...,...,...
84,0.831514,153.0
85,0.831514,98.0
86,0.831514,37.0
87,0.831514,63.0


Side-by-side, they don't look great. Can you improve them?

<br><br>

## EXERCISE #1
Try increasing the number of epochs to 1,000 (when the model is fairly well trained then the results printed for each 10x epochs will be fairly stable and not change much). Does this give better results?

<br><br>

## EXERCISE #2 (optional)
Try experimenting with the architecture (number of neurons and/or number of layers). Can we reach an optimal architecture?

In [ ]:
import torch

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training loop with 1000 epochs
epochs = 1000

print(f"\nStarting training with {epochs} epochs...")

for epoch in range(epochs):
  for inputs, targets in train_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    model.to(device)

    model.train()
    optimiser.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimiser.step()

  if (epoch+1) % 100 == 0: # Print loss every 100 epochs for brevity
      print(f'Epoch [{epoch+1}/{epochs}], Loss: {round(loss.item(), 4)}')

print("\nTraining complete!")



Starting training with 1000 epochs...
Epoch [100/1000], Loss: 12019.2568
Epoch [200/1000], Loss: 15867.1875
Epoch [300/1000], Loss: 54998.8203
Epoch [400/1000], Loss: 7154.0898
Epoch [500/1000], Loss: 32376.9961
Epoch [600/1000], Loss: 5622.2188
Epoch [700/1000], Loss: 33005.7578
Epoch [800/1000], Loss: 31499.7402
Epoch [900/1000], Loss: 38965.1719
Epoch [1000/1000], Loss: 34977.6055

Training complete!


Now, let's re-evaluate the model's performance after training for 1,000 epochs with target scaling.

In [ ]:
# Evaluation
model.eval()
predictions_1000_epochs = []
actuals_1000_epochs = []
mse_values_1000_epochs_scaled = [] # Store MSE for scaled values
mse_values_1000_epochs_unscaled = [] # Store MSE for unscaled values

with torch.no_grad():
    for inputs, targets_scaled in test_loader: # Renamed targets to targets_scaled
        inputs, targets_scaled = inputs.to(device), targets_scaled.to(device)
        outputs_scaled = model(inputs)

        # Inverse transform to original scale
        outputs_unscaled = torch.tensor(scaler_y.inverse_transform(outputs_scaled.cpu().numpy()), dtype=torch.float32)
        targets_unscaled = torch.tensor(scaler_y.inverse_transform(targets_scaled.cpu().numpy()), dtype=torch.float32)

        predictions_1000_epochs.extend(outputs_unscaled.flatten().cpu().numpy()) # Flatten and convert to numpy
        actuals_1000_epochs.extend(targets_unscaled.flatten().cpu().numpy()) # Flatten and convert to numpy

        # Calculate MSE for scaled and unscaled
        mse_scaled = criterion(outputs_scaled, targets_scaled)
        mse_values_1000_epochs_scaled.append(mse_scaled.item())

        mse_unscaled = criterion(outputs_unscaled, targets_unscaled) # Calculate MSE on unscaled values
        mse_values_1000_epochs_unscaled.append(mse_unscaled.item())

# Calculate and print the average MSE (for unscaled values)
avg_mse_1000_epochs_unscaled = np.mean(mse_values_1000_epochs_unscaled)
print(f"Average MSE on test set (original scale) after 1000 epochs: {avg_mse_1000_epochs_unscaled}")

# Optional: print scaled MSE as well
avg_mse_1000_epochs_scaled = np.mean(mse_values_1000_epochs_scaled)
print(f"Average MSE on test set (scaled) after 1000 epochs: {avg_mse_1000_epochs_scaled}")


# Create DataFrame
results_df_1000_epochs = pd.DataFrame({'Predicted': predictions_1000_epochs, 'Actual': actuals_1000_epochs})
display(results_df_1000_epochs)


NameError: name 'scaler_y' is not defined

Now, let's re-evaluate the model's performance after training for 1,000 epochs.

I embarked on building a Dense Neural Network using PyTorch for a regression task on the Diabetes dataset, starting with ensuring GPU availability and meticulously preparing the data by converting it to PyTorch tensors, performing train/test splits, and setting up DataLoaders for efficient processing. We then designed a DiabetesModel featuring hidden layers with ReLU activation and an MSELoss criterion, which we trained using the Adam optimizer. Initially, after 100 epochs, we observed a high Mean Squared Error (MSE) and noticed the model's predictions were almost constant, indicating a critical learning issue. To resolve this, we introduced StandardScaler to appropriately scale the target variable y, and after retraining the model for 1,000 epochs with this scaled target, we performed an inverse transformation on the predictions during evaluation to derive more meaningful results in the original data scale, aiming for improved and accurate model performance.

